# Manufacturing Process Parameter Optimization Using CMA-ES

In this notebook, we'll explore how to use the Covariance Matrix Adaptation Evolution Strategy (CMA-ES) to optimize manufacturing process parameters. This is a common industrial engineering problem where we need to balance multiple competing objectives.

CMA-ES is a powerful evolutionary algorithm for continuous parameter optimization, especially useful when:
- The objective function is non-linear or non-convex
- There are multiple local optima
- The function is noisy or complex

## Problem Description

We'll simulate a manufacturing process with three key parameters:
1. **Temperature** (°C): Acceptable range 150-250°C
2. **Pressure** (MPa): Acceptable range 5-15 MPa
3. **Processing time** (min): Acceptable range 10-30 minutes

We need to optimize these parameters to achieve multiple objectives:
- **Maximize production rate** (units/hour)
- **Minimize defect rate** (%)
- **Minimize energy consumption** (kWh)

Let's start by importing the necessary libraries and defining our objective functions.

In [ ]:
import numpy as np
import cma
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

## Defining the Objective Functions

We'll define three objective functions, one for each goal we're trying to optimize:

1. **Production Rate**: We want to maximize this, so we'll negate the function since CMA-ES minimizes by default
2. **Defect Rate**: We want to minimize this (lower is better)
3. **Energy Consumption**: We want to minimize this (lower is better)

Each function will include penalties for solutions outside the acceptable parameter ranges.

In [ ]:
# Manufacturing process parameter optimization example
# Objectives:
# 1. Maximize production rate (units/hour)
# 2. Minimize defect rate (%)
# 3. Minimize energy consumption (kWh)

def manufacturing_process(x):
    """
    Simulates a manufacturing process with the following parameters:
    x[0] = Temperature (°C), acceptable range: [150, 250]
    x[1] = Pressure (MPa), acceptable range: [5, 15]
    x[2] = Processing time (min), acceptable range: [10, 30]
    
    Returns the negative of production rate (we'll minimize, so negative means maximize production)
    """
    temperature, pressure, process_time = x
    
    # Penalize solutions outside acceptable ranges
    penalty = 0
    if temperature < 150 or temperature > 250:
        penalty += 1000
    if pressure < 5 or pressure > 15:
        penalty += 1000
    if process_time < 10 or process_time > 30:
        penalty += 1000
    
    # Production rate model (units/hour)
    # Higher temperature and pressure increase production rate
    # Longer processing time decreases production rate
    production_rate = (
        20 + 0.1 * (temperature - 150) 
        + 2 * (pressure - 5) 
        - 0.8 * (process_time - 10)
    )
    
    # Add some interesting non-linearity
    production_rate += 5 * np.sin(temperature/25) * np.cos(pressure**2/9)
    
    # Add penalty
    return -production_rate + penalty  # Negative because we're minimizing


In [ ]:
def defect_rate(x):
    """
    Simulates defect rate for the manufacturing process.
    Returns defect percentage.
    """
    temperature, pressure, process_time = x
    
    # Penalize solutions outside acceptable ranges
    penalty = 0
    if temperature < 150 or temperature > 250:
        penalty += 100
    if pressure < 5 or pressure > 15:
        penalty += 100
    if process_time < 10 or process_time > 30:
        penalty += 100
    
    # Base defect rate
    defect = 5.0
    
    # Temperature affects defect rate with a sweet spot around 200°C
    defect += 0.05 * (temperature - 200)**2 / 25
    
    # Pressure affects defect rate with a sweet spot around 10 MPa
    defect += 0.5 * (pressure - 10)**2
    
    # Longer processing time generally reduces defects
    defect -= 0.1 * (process_time - 10)
    
    # Add some non-linearity
    defect += 2 * np.sin(temperature**3/(20**3)) * np.sin(pressure/2)
    
    return np.minimum(defect + penalty, 100)

In [ ]:
def energy_consumption(x):
    """
    Simulates energy consumption for the manufacturing process.
    Returns energy consumption in kWh.
    """
    temperature, pressure, process_time = x
    
    # Penalize solutions outside acceptable ranges
    penalty = 0
    if temperature < 150 or temperature > 250:
        penalty += 1000
    if pressure < 5 or pressure > 15:
        penalty += 1000
    if process_time < 10 or process_time > 30:
        penalty += 1000
    
    # Base energy consumption
    energy = 10.0
    
    # Higher temperature increases energy consumption
    energy += 0.2 * (temperature - 150)
    
    # Higher pressure increases energy consumption
    energy += 1.5 * (pressure - 5)
    
    # Longer processing time increases energy consumption
    energy += 0.5 * process_time
    
    return energy + penalty


## Multi-Objective Optimization

In industrial engineering, we often need to balance multiple competing objectives. There are several approaches to handle this:

1. **Weighted Sum**: Combine objectives with different weights based on their importance
2. **Pareto Front**: Find a set of non-dominated solutions
3. **Lexicographic**: Optimize objectives in order of priority

For simplicity, we'll use the weighted sum approach here, but you could extend this to find the Pareto front or use other multi-objective techniques.


In [ ]:
def multi_objective(x, weights=[0.5, 0.3, 0.2]):
    """
    Combines multiple objectives using weighted sum approach.
    weights: [production_weight, defect_weight, energy_weight]
    """
    # Notice the negative on production rate is already applied in its function
    return (
        weights[0] * manufacturing_process(x) + 
        weights[1] * defect_rate(x) + 
        weights[2] * energy_consumption(x)
    )

## Single-Objective Optimization

Let's first optimize for just the production rate to see how CMA-ES works on a single objective.

CMA-ES requires:
- An initial solution (x0)
- An initial step size (sigma0)
- An objective function to minimize

The algorithm will then explore the solution space, adapting its search distribution based on successful solutions it finds.


In [ ]:
# Run single-objective optimization for production rate
def optimize_production():
    print("Optimizing for production rate...")
    x0 = [200, 10, 20]  # Initial guess [temperature, pressure, process_time]
    sigma0 = 20  # Initial step size
    
    # Run CMA-ES optimization
    result = cma.fmin(manufacturing_process, x0, sigma0, 
                       options={'maxfevals': 1000, 'verb_disp': 1})
    
    # Extract the best solution
    best_params = result[0]
    best_production = -manufacturing_process(best_params)  # Convert back to positive
    best_defect = defect_rate(best_params)
    best_energy = energy_consumption(best_params)
    
    print(f"Best parameters: Temperature={best_params[0]:.2f}°C, "
          f"Pressure={best_params[1]:.2f}MPa, Process time={best_params[2]:.2f}min")
    print(f"Production rate: {best_production:.2f} units/hour")
    print(f"Defect rate: {best_defect:.2f}%")
    print(f"Energy consumption: {best_energy:.2f} kWh")
    
    return best_params, best_production, best_defect, best_energy

Now let's optimize all three objectives simultaneously using our weighted sum approach. 
By adjusting the weights, we can emphasize different objectives based on their importance to the manufacturing process.

In [ ]:
def optimize_multi_objective(weights=[0.5, 0.3, 0.2]):
    print(f"Multi-objective optimization with weights: {weights}")
    x0 = [200, 10, 20]  # Initial guess [temperature, pressure, process_time]
    sigma0 = 20  # Initial step size
    
    # Create an objective function with fixed weights
    def objective(x):
        return multi_objective(x, weights)
    
    # Run CMA-ES optimization
    result = cma.fmin(objective, x0, sigma0, 
                       options={'maxfevals': 1000, 'verb_disp': 1})
    
    # Extract the best solution
    best_params = result[0]
    best_production = -manufacturing_process(best_params)  # Convert back to positive
    best_defect = defect_rate(best_params)
    best_energy = energy_consumption(best_params)
    
    print(f"Best parameters: Temperature={best_params[0]:.2f}°C, "
          f"Pressure={best_params[1]:.2f}MPa, Process time={best_params[2]:.2f}min")
    print(f"Production rate: {best_production:.2f} units/hour")
    print(f"Defect rate: {best_defect:.2f}%")
    print(f"Energy consumption: {best_energy:.2f} kWh")
    
    return best_params, best_production, best_defect, best_energy


## Visualizing the Objective Functions

To better understand our optimization problem, let's visualize each objective function.
We'll create 3D surface plots showing how the objectives change with temperature and pressure
(keeping process time fixed at 20 minutes for visualization purposes).

This visualization helps us understand:
- The shape and complexity of each objective function
- Potential local optima and challenges for optimization
- The conflicting nature of our multiple objectives

In [ ]:
# Visualize the production rate objective function
def visualize_objective(objective_func, title):
    # Create a grid of temperature and pressure values (fixing process_time at 20)
    temp_range = np.linspace(150, 250, 50)
    pressure_range = np.linspace(5, 15, 50)
    T, P = np.meshgrid(temp_range, pressure_range)
    Z = np.zeros_like(T)
    
    # Calculate objective function value at each grid point
    for i in range(len(pressure_range)):
        for j in range(len(temp_range)):
            if objective_func == manufacturing_process:
                Z[i, j] = -objective_func([T[i, j], P[i, j], 20])  # Negate to show production
            else:
                Z[i, j] = objective_func([T[i, j], P[i, j], 20])
    
    # Create 3D plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(T, P, Z, cmap=cm.viridis, alpha=0.8)
    
    # Add labels and colorbar
    ax.set_xlabel('Temperature (°C)')
    ax.set_ylabel('Pressure (MPa)')
    if objective_func == manufacturing_process:
        ax.set_zlabel('Production Rate')
        ax.set_title(f'{title} (Process Time = 20min)')
    elif objective_func == defect_rate:
        ax.set_zlabel('Defect Rate (%)')
        ax.set_title(f'{title} (Process Time = 20min)')
    elif objective_func == defect_rate:
        ax.set_zlabel('Energy Consumption (kWh)')
        ax.set_title(f'{title} (Process Time = 20min)')
    else:
        ax.set_zlabel('Objective')
        ax.set_title(title)
    
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)
    plt.tight_layout()
    plt.show()

## Running the Optimization and Comparing Results

Now let's run both our single-objective and multi-objective optimizations and compare the results.
This will help us understand the trade-offs involved in optimizing for multiple competing objectives.

For the multi-objective case, we'll use weights that prioritize:
- Production rate (40%)
- Defect rate (30%)
- Energy consumption (30%)

In [ ]:
# Run single-objective optimization for production rate
best_params_prod, best_production, best_defect_prod, best_energy_prod = optimize_production()

Now let's run the multi-objective optimization with differently balanced weights:

In [ ]:
# Run multi-objective optimization with custom weights
weights = [0.4, 0.5, 0.1]  # Production, Defect, Energy
best_params_multi, best_production_multi, best_defect_multi, best_energy_multi = optimize_multi_objective(weights)

Let's compare the results from our two optimization approaches:

In [ ]:
print("Comparison:")
print(f"Production optimization: {best_production:.2f} units/hour, {best_defect_prod:.2f}% defects, {best_energy_prod:.2f} kWh")
print(f"Multi-objective optimization: {best_production_multi:.2f} units/hour, {best_defect_multi:.2f}% defects, {best_energy_multi:.2f} kWh")

<div class="alert alert-block alert-success">
<h2>Exercise: Visualizing Each Objective Function</h2>

Visualize each objective function to better understand the optimization landscape. Then, visualize the multi-objective landscape with a fixed value of weights. When you change the weights, how does the optimization landscape change?
</div>

In [ ]:
visualize_objective(multi_objective, "Manufacturing Process Objective")

## Exploring Different Weight Combinations

In industrial engineering, the importance of different objectives may vary based on business priorities.
Let's explore how different weight combinations affect our optimal solution.

In [ ]:
def explore_weight_combinations(weight_combinations):
    # Define different weight combinations to explore
   
    results = []
    
    for weights in weight_combinations:
        print(f"\nOptimizing with weights: {weights}")
        best_params, production, defect, energy = optimize_multi_objective(weights)
        results.append((weights, best_params, production, defect, energy))
    
    # Display summary table
    print("\nWeight Combination Results Summary:")
    print("="*80)
    print("{:<30} {:<15} {:<15} {:<15}".format(
        "Weights [Prod, Defect, Energy]", "Production", "Defect Rate", "Energy"))
    print("-"*80)
    
    for weights, _, prod, defect, energy in results:
        weight_str = f"[{weights[0]:.1f}, {weights[1]:.1f}, {weights[2]:.1f}]"
        print("{:<30} {:<15.2f} {:<15.2f} {:<15.2f}".format(
            weight_str, prod, defect, energy))
    return results

In [ ]:
def plot_weight_results(results):
    # Plot the results
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Normalize each metric for better visualization
    max_prod = max(result[2] for result in results)
    max_defect = max(result[3] for result in results)
    max_energy = max(result[4] for result in results)
    
    # Create x positions for the bars
    x = np.arange(len(results))
    width = 0.25  # width of the bars
    
    # Plot bars for each metric
    rects1 = ax.bar(x - width, [result[2]/max_prod for result in results], width, label='Production Rate')
    rects2 = ax.bar(x, [1 - result[3]/max_defect for result in results], width, label='Quality (1-Defect)')
    rects3 = ax.bar(x + width, [1 - result[4]/max_energy for result in results], width, label='Energy Efficiency')
    
    # Add labels and legend
    ax.set_ylabel('Normalized Performance')
    ax.set_title('Optimization Results with Different Weight Combinations')
    ax.set_xticks(x)
    ax.set_xticklabels([f"W{i+1}" for i in range(len(results))])
    ax.legend()
    
    # Add a text box explaining the weight combinations
    textstr = '\n'.join([
        f"W1: Production-focused {results[0][0]}",
        f"W2: Quality-focused {results[1][0]}",
        f"W3: Energy-focused {results[2][0]}",
        f"W4: Balanced {results[3][0]}"
    ])
    
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.05, textstr, transform=ax.transAxes, fontsize=10,
            verticalalignment='bottom', bbox=props)
    
    plt.tight_layout()
    plt.show()

<div class="alert alert-block alert-success">
<h2>Exercise: Different weight combinations</h2>

Compare different weight combinations. Each weight combination should sum to 1. Try configurations that heavily prioritizes production, quality, and energy efficiency, as well as one that is equal. Using the above functions, solve these combinations using CMA-ES and plot the results.</div>

In [ ]:
weight_combinations = [
    [0.8, 0.1, 0.1],  # Heavily prioritize production
    # FILL IN HERE
]

In [ ]:
results = explore_weight_combinations(weight_combinations)

In [ ]:
plot_weight_results(results)